<a href="https://colab.research.google.com/github/jamescyl/myrepo/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install langchain
!pip install google-generativeai langchain-google-genai
!pip install wikipedia

In [2]:
#Bring in deps
import os
import sys
sys.path.append('/content/drive/MyDrive')
from apikey import apikey

os.environ['API_KEY'] = apikey
GOOGLE_API_KEY = apikey



In [ ]:

#import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.utilities import WikipediaAPIWrapper


os.environ['API_KEY'] = apikey
GOOGLE_API_KEY = apikey

# App framework
#st.title('YouTube GPT Creator')
# Insert the streamlit run command
#!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py st_view.py
#prompt = st.text_input('Plug in your prompt here')

my_prompt = 'Wonderful world'

# Prompt templates
title_template = PromptTemplate(
    input_variables = ['topic'],
    template='write me a youtube video title about {topic}'
)

script_template = PromptTemplate(
    input_variables = ['title', 'wikipedia_research'],
    template='write me a youtube video script based on this title TITLE: {title} while leveraging this wikipedia reserch:{wikipedia_research} '
)

# Memory
title_memory = ConversationBufferMemory(input_key='topic', memory_key='chat_history')
script_memory = ConversationBufferMemory(input_key='title', memory_key='chat_history')

# Llms
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True, output_key='title', memory=title_memory)
script_chain = LLMChain(llm=llm, prompt=script_template, verbose=True, output_key='script', memory=script_memory)

wiki = WikipediaAPIWrapper()

# Show stuff to the screen if there's a prompt
if my_prompt:
    title = title_chain.run(my_prompt)
    wiki_research = wiki.run(my_prompt)
    script = script_chain.run(title=title, wikipedia_research=wiki_research)

print(my_prompt)
print(title_memory.buffer)
print(script_memory.buffer)

"""
    st.write(title)
    st.write(script)

    with st.expander('Title History'):
        st.info(title_memory.buffer)

    with st.expander('Script History'):
        st.info(script_memory.buffer)

    with st.expander('Wikipedia Research'):
        st.info(wiki_research)
"""


#llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
#tweet_prompt = PromptTemplate.from_template("You are a content creator. Write me a tweet about {topic}.")
#tweet_chain = LLMChain(llm=llm, prompt=tweet_prompt, verbose=True)
#if __name__=="__main__":
#    topic = "what is streamlit?"
#    resp = tweet_chain.run(topic=topic)
#    print(resp)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

image_url = '/content/drive/MyDrive/SAM_0147.JPG'

import matplotlib.pyplot as plt
from PIL import Image

# Load the image
image = Image.open(image_url)

# Display the image
plt.imshow(image)
plt.show()


# Llms
llm = ChatGoogleGenerativeAI(model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY)
message = HumanMessage(
    content = [
        {
            "type": "text",
            "text": "What's in this image?",
        }, # You can optionally provide text paths
        {"type": "image_url","image_url": image_url},
    ]
)
llm.invoke([message])


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
embeddings.embed_query("hello, world!")

In [ ]:
!pip install google-cloud-core


In [ ]:
import google.cloud

# Create a client
client = google.cloud.vectorstore.v1.VectorStoreServiceClient()

# Set the project ID and region
project_id = "YOUR_PROJECT_ID"
region = "YOUR_REGION"  # e.g., "us-central1"

# Set the Vector Store ID
vector_store_id = "YOUR_VECTOR_STORE_ID"

# Set the Vector Store spec
spec = {"dimensionality": 512}

# Create the Vector Store
vector_store = client.create_vector_store(
    parent=f"projects/{project_id}/locations/{region}",
    vector_store_id=vector_store_id,
    vector_store=spec,
)

# Print the Vector Store name
print(f"Vector Store created: {vector_store.name}")


In [42]:
from google.cloud import GoogleVectorStore

# Create a new store for housing your documents.
corpus_store = GoogleVectorStore.create_corpus(display_name="My Corpus")

# Create a new document under the above corpus.
document_store = GoogleVectorStore.create_document(
    corpus_id=corpus_store.corpus_id, display_name="My Document"
)

# Upload some texts to the document.
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
for file in DirectoryLoader(path="data/").load():
    documents = text_splitter.split_documents([file])
    document_store.add_documents(documents)

# Talk to your entire corpus with possibly many documents.
aqa = corpus_store.as_aqa()
answer = aqa.invoke("What is the meaning of life?")

# Read the response along with the attributed passages and answerability.
print(response.answer)
print(response.attributed_passages)
print(response.answerable_probability)

ImportError: cannot import name 'GoogleVectorStore' from 'google.cloud' (unknown location)